In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

# Display all rows and columns when printing DataFrames
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [3]:
# Features and target
features = [
    "age", "physical_activity_minutes_per_week", "alcohol_consumption_per_week",
    "diet_score", "sleep_hours_per_day", "screen_time_hours_per_day", "bmi",
    "waist_to_hip_ratio", "systolic_bp", "diastolic_bp", "heart_rate",
    "cholesterol_total", "hdl_cholesterol", "ldl_cholesterol", "triglycerides",
    'gender', 'ethnicity', 'education_level', 'income_level',
    'smoking_status', 'employment_status',
    "family_history_diabetes", "hypertension_history", "cardiovascular_history"
]

target = "diagnosed_diabetes"


In [4]:
# Separate X and y
cat_cols = ['gender', 'ethnicity', 'education_level', 'income_level', 'smoking_status', 'employment_status']
X_train_raw = train[features]
y_train = train[target]

In [5]:
# One-hot encode categorical features
X_train = pd.get_dummies(X_train_raw, columns=cat_cols, drop_first=True)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Initialize baseline model
baseline_model = LogisticRegression()

# 5-fold cross-validation ROC-AUC
roc_auc_scores = cross_val_score(
    baseline_model, X_train, y_train, cv=5, scoring='roc_auc'
)

/Users/jose/Documents/2025 Kaggle Playground Competitions/playground-series-s5e12/playground-series-s5e12/kaggle_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/jose/Documents/2025 Kaggle Playground Competitions/playground-series-s5e12/playground-series-s5e12/kaggle_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-le

Cross-validated ROC-AUC per fold: [0.66866294 0.66508602 0.67367386 0.66888086 0.67341448]
Mean CV ROC-AUC: 0.6699436328412771


/Users/jose/Documents/2025 Kaggle Playground Competitions/playground-series-s5e12/playground-series-s5e12/kaggle_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
print("Cross-validated ROC-AUC per fold:", roc_auc_scores)
print("Mean CV ROC-AUC:", roc_auc_scores.mean())


Cross-validated ROC-AUC per fold: [0.66866294 0.66508602 0.67367386 0.66888086 0.67341448]
Mean CV ROC-AUC: 0.6699436328412771


In [7]:
baseline_model.fit(X_train, y_train)

/Users/jose/Documents/2025 Kaggle Playground Competitions/playground-series-s5e12/playground-series-s5e12/kaggle_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [8]:
# One-hot encode test set the same way as training set
X_test_raw = test[features]
X_test = pd.get_dummies(X_test_raw, columns=cat_cols, drop_first=True)

# Ensure test has same columns as training (add missing columns with 0)
for col in X_train.columns:
    if col not in X_test.columns:
        X_test[col] = 0
X_test = X_test[X_train.columns]  # reorder columns

# Predict probabilities for class 1
y_test_proba = baseline_model.predict_proba(X_test)[:, 1]

In [10]:
y_test_proba

array([0.64393525, 0.6955198 , 0.62120598, ..., 0.50808852, 0.5547028 ,
       0.54521364])

In [11]:
# Save Predictions in the required format
submission = pd.DataFrame({
    "id": test["id"],                # test set IDs
    "diagnosed_diabetes": y_test_proba  # rename column to match required format
})

submission.to_csv("../Submissions/Logistic_Regression.csv", index=False)


In [12]:
submission.head()

,id,diagnosed_diabetes
0,700000,0.643935
1,700001,0.695520
2,700002,0.621206
3,700003,0.637718
4,700004,0.778737
